In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
torch.manual_seed(2333)

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28]) torch.float32
Shape of y: torch.Size([64]) torch.int64


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [6]:
def train(dataloader, model, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = F.nll_loss(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss +=  F.nll_loss(pred, y, reduction='sum').item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
device = torch.device("cpu")
# torch.set_num_threads(1) 

In [8]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, optimizer)
    test(test_dataloader, model)
    scheduler.step()
print("Done!")

Epoch 1
-------------------------------
loss: 2.295813  [    0/60000]
loss: 0.508335  [ 6400/60000]
loss: 0.420510  [12800/60000]
loss: 0.496305  [19200/60000]
loss: 0.382754  [25600/60000]
loss: 0.457507  [32000/60000]
loss: 0.282513  [38400/60000]
loss: 0.520273  [44800/60000]
loss: 0.547393  [51200/60000]
loss: 0.319387  [57600/60000]
Test Error: 
 Accuracy: 89.5%, Avg loss: 19.236168 

Epoch 2
-------------------------------
loss: 0.190863  [    0/60000]
loss: 0.304843  [ 6400/60000]
loss: 0.200837  [12800/60000]
loss: 0.367021  [19200/60000]
loss: 0.298409  [25600/60000]
loss: 0.352417  [32000/60000]
loss: 0.171559  [38400/60000]
loss: 0.347360  [44800/60000]
loss: 0.291345  [51200/60000]
loss: 0.305732  [57600/60000]
Test Error: 
 Accuracy: 90.9%, Avg loss: 16.266098 

Epoch 3
-------------------------------
loss: 0.235274  [    0/60000]
loss: 0.293353  [ 6400/60000]
loss: 0.133521  [12800/60000]
loss: 0.258932  [19200/60000]
loss: 0.243643  [25600/60000]
loss: 0.425698  [32000/6

In [10]:
input_names = ["image"]
output_names = ["prediction"]
dummy_input = torch.randn(10000, 1, 28, 28)
torch.onnx.export(model,
                  dummy_input,
                  "fmnist.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [11]:
torch.set_num_interop_threads(1)
dummy_input = torch.rand(10000, 1, 28, 28, dtype=torch.float32, device=device)

In [12]:
%%time

with torch.no_grad():
    y = model(dummy_input)
y.shape

CPU times: user 9.63 s, sys: 7.39 s, total: 17 s
Wall time: 1.65 s


torch.Size([10000, 10])

In [13]:
import numpy as np

np_input = dummy_input.detach().numpy()
with open('dummy_64_fminist_in.dat', 'wb') as f:
    np_input.tofile(f)
    
np_output = y.detach().numpy()
with open('dummy_64_fminist_out.dat', 'wb') as f:
    np_output.tofile(f)
